In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from joblib import dump , load
import plotly.express as px

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/digipodium/Datasets/main/house_pricing.csv")
df.info()
df.head(2)

In [ ]:
X = df[['Beds','Baths','SquareFeet']]
y = df['Price']
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=.2,random_state=1)

In [ ]:
print("random Forest")
model2 = RandomForestRegressor(max_depth=25)
model2.fit(xtrain,ytrain)
print("score:",model2.score(xtest,ytest)*100)
pred = model2.predict(X)
print("mse:",mean_squared_error(y,pred))
print("mse:",mean_absolute_error(y,pred))


In [ ]:
forest_score = cross_val_score(model2,X,y,cv=6)
print(forest_score,f"average:{forest_score.mean():.2f}",f"std:{forest_score.std():.2f}")

GRIDSEARCHCV

In [ ]:
params ={ 
    'n_estimators' : list(range(100,501,200)),
    'criterion' :["squared_error","absolute_error","poisson"],
    'max_depth': list(range(5,51,25)),

}
params

In [ ]:
grid = GridSearchCV(estimator=RandomForestRegressor(),param_grid=params,cv=3,n_jobs=-1,verbose=2)


In [ ]:
grid.fit(X,y)

In [ ]:
gf=pd.DataFrame(grid.cv_results_)

In [ ]:
gf.sort_values(by='rank_test_score',inplace=True)

In [ ]:
gf

In [ ]:
grid.best_estimator_

In [13]:
dump(grid.best_estimator_,"house_pricing_model_73.pkl")

['house_pricing_model_73.pkl']